# Gerando IPTU espacializado por pontos e agregados por distrito para todos exercícios

In [12]:
import vaex
import pandas as pd
import geopandas as gpd
from shapely.validation import make_valid
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
import os

In [13]:
df = vaex.open(f'data/IPTU_2018/IPTU_2018.hdf5')#.to_pandas_df()
gdf_distrito = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')

In [14]:
df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
df['sq'] = df.setor + df.quadra
df['sqlc'] = df.sqlc

In [15]:
df = df.to_pandas_df()

In [16]:
for i, distrito in gdf_distrito.iterrows():
    
    print(distrito.ds_nome)
    ano = '2018'
    d_nome, d_codigo = distrito.ds_nome.replace(" ", "_"), distrito.ds_codigo.zfill(2)
    result_file = f'iptu_por_pontos/{ano}/SIRGAS_IPTU_{ano}_{d_codigo}_{d_nome}.gpkg'

    if os.path.exists(result_file):
        continue



    gdf_lote = gpd.read_file(f'zip://data/lotes/SIRGAS_SHP_LOTES_{d_codigo}_{d_nome}.zip!SIRGAS_SHP_LOTES_{d_codigo}_{d_nome}/SIRGAS_SHP_LOTES_{d_codigo}_{d_nome}.shp')
    gdf_lote = gdf_lote[gdf_lote.lo_tp_lote == 'F']
    gdf_lote['sqlc'] = gdf_lote.lo_setor + gdf_lote.lo_quadra + gdf_lote.lo_lote.where(gdf_lote["lo_condomi"] == '00', '0000') + gdf_lote.lo_condomi
    gdf_lote['geometry'] = gdf_lote.representative_point()

    df_sqlc = df.merge(gdf_lote, how='left', on='sqlc')

    gdf_lote['sq'] = gdf_lote.lo_setor + gdf_lote.lo_quadra
    gdf_lote = gdf_lote.dissolve(by=['sq'])
    gdf_lote.geometry = gdf_lote.representative_point()
    df_sq = df[df_sqlc.lo_lote.isna()].merge(gdf_lote, how='left', on='sq')

    columns = df.columns.to_list()
    columns.append('geometry')

    df_sqlc = df_sqlc.loc[df_sqlc.lo_lote.notna(), columns]
    df_sqlc["espacializado_por_lote"] = True
    df_sq.rename(columns={'sqlc_x':'sqlc'}, inplace=True)
    df_sq = df_sq.loc[df_sq.lo_lote.notna(), columns]
    df_sq["espacializado_por_lote"] = False

    df_lotes = pd.concat([df_sqlc, df_sq]).reset_index(drop=True)
    gpd.GeoDataFrame(df_lotes).to_file(result_file, driver='GPKG', crs='EPSG:31983')

    # break

## TODO - criar interação sobre cada ano

PIRITUBA


In [ ]:
import glob

In [8]:
iptus = glob.glob('iptu_por_pontos/2018/*')

In [9]:
df_iptus = []

for iptu in iptus:
    gdf = gpd.read_file(iptu)
    df_iptus.append(gdf)

In [10]:
df_iptus_concat = pd.concat(df_iptus)

In [11]:
gdf_iptus = gpd.GeoDataFrame(df_iptus_concat.reset_index())

In [12]:
gdf_iptus.to_file('tmp/IPTUs-2018-pontos.gpkg', driver='GPKG')

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
